Assignment 2 - part 1

In [ ]:
# Import necessary libraries

import requests
import lxml.html as lh
import pandas as pd



In [ ]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [ ]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

In [ ]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))


Creating Pandas DataFrame 

In [ ]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [ ]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

In [ ]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [ ]:
# Access the top 5 rows of the data frame 
df.head()


Rearranging and renaming the columns

In [ ]:
df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()


Cleaning the messy string in the Borough column

In [ ]:
df = df.replace('\n',' ', regex=True)
df.head()

Dropping all cells with a borough that is Not assigned

In [ ]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

Combining Neighbourhoods based on similar Postcode and Borough

In [ ]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)


Removing any space in the start of the string

In [ ]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()


Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"

In [ ]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [ ]:
# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']